## Latency calculation with ENOT

This notebook describes how to calculate latency using ENOT framework.

### Main chapters of this notebook:
1. Initialize latency of search space (`SearchSpaceModel`)
1. Calculate latency of arbitrary model/module
1. Fill `SearchSpaceModel` with precalculated latency
1. Save/Load `SearchSpaceLatencyContainer`

## Initialize latency of search space (`SearchSpaceModel`)

To initialize latency of `SearchSpaceModel` import `SearchSpaceModel` from `enot.models` and `initialize_latency` function from `enot.latency`:

In [ ]:
from enot.models import SearchSpaceModel
from enot.latency import initialize_latency

`initialize_latency` has the following signature:

```python
def initialize_latency(
    latency_type: str,
    search_space: SearchSpaceModel,
    inputs: Tuple,
    keyword_inputs: Optional[Dict[str, Any]] = None,
) -> SearchSpaceLatencyContainer
```

`latency_type (str)` — type of the latency to be initialized in `search_space`.
Now ENOT supports only multiply-accumulate (MAC) latency type.
For MAC latency initialization use `latency_type='mmac'`.

For most modules ENOT has built-in MAC calculator, but for unsupported modules it is possible to use third-party calculators:

- to use **PyTorch-OpCounter (thop)** third-party MAC calculator pass `latency_type='mmac.thop'`
- to use **PyTorch-estimate-flops (pthflops)** third-party MAC calculator pass `latency_type='mmac.pthflops'`

Note: third-party calculators complement built-in calculator, i.e. if built-in calculator knows how to calculate latency of module, then third-party calculator will not be used for this module.

`search_space` — `SearchSpaceModel` for latency calculation.

        
`inputs: Tuple` — `search_space` input.

Also *keyword arguments* can be passed.


`initialize_latency` returns `SearchSpaceLatencyContainer`, that can be used to calculate statistics or visualization of latency.

For example, let us calculate MAC-latency of search space from <span style="color:green;white-space:nowrap">***1. Tutorial - getting started***</span>:

In [ ]:
import torch
from enot.models.mobilenet import build_mobilenet

In [ ]:
model = build_mobilenet(
    search_ops=['MIB_k=3_t=6', 'MIB_k=5_t=6', 'MIB_k=7_t=6'],
    num_classes=10,
    blocks_out_channels=[24, 32, 64, 96, 160, 320],
    blocks_count=[2, 2, 2, 1, 2, 1],
    blocks_stride=[2, 2, 2, 1, 2, 1],
)
search_space = SearchSpaceModel(model).cpu()
inputs = torch.ones(1, 3, 244, 224)

Now MAC-latency of `search_space` can be initialized in the following way:

In [ ]:
initialize_latency('mmac', search_space, (inputs, ));  # ; suppress output of statistics.

Or we can enable **PyTorch-OpCounter** third-party calculator and print some statistics:

In [ ]:
from enot.latency import min_latency
from enot.latency import mean_latency
from enot.latency import max_latency
from enot.latency import median_latency
from enot.latency import current_latency

In [ ]:
container = initialize_latency('mmac.thop', search_space, (inputs, ))
print(f'Constant latency: {container.constant_latency}\n'
      f'Min latency: {min_latency(container)}\n'
      f'Mean latency: {mean_latency(container)}\n'
      f'Max latency: {max_latency(container)}\n'
      f'Median latency: {median_latency(container)}\n')

Container can be visualized as a heatmap, cast to string or printed

In [ ]:
from enot.visualization.latency import plot_latency_heatmap
plot_latency_heatmap(container, annotate_values=True, figsize=(8, 8));

In [ ]:
print(container)

To get latency of `search_space`:

In [ ]:
latency = current_latency(search_space)
print(f'Latency: {latency}')

To reset latency of `search_space` use `reset_latency` from `enot.latency`:

In [ ]:
from enot.latency import reset_latency

In [ ]:
reset_latency(search_space)

In [ ]:
search_space.latency_type == None

## Calculate latency of arbitrary model

To calculate latency of arbitary model/module import `MacCalculatorThop` or `MacCalculatorPthflops` from `enot.latency`

In [ ]:
from enot.latency import MacCalculatorThop
from enot.latency import MacCalculatorPthflops

Latency calculators have only one function with the following signature:

```python
def calculate(
    model: nn.Module,
    inputs: Tuple,
    ignore_modules: Optional[List[Type[nn.Module]]] = None,
    **options
) -> float:
```

So you can pass model, inputs and list of modules that you want to ignore in calculation as well as some additional options.

For example (model and inputs from previous example are used):

In [ ]:
MacCalculatorThop().calculate(model, inputs)

In [ ]:
# MacCalculatorPthflops().calculate(model, inputs)

## Fill SearchSpaceModel with precalculated latency

For example, there are precalculated constant latency (static part of search space) and latencies of operations as **numpy** array:

In [ ]:
import numpy as np
from enot.latency import SearchSpaceLatencyContainer

In [ ]:
precalc_constant_latency = 10
precalc_operations_latencies = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [1, 2, 3],
])

To initialize `SearchSpaceModel` with these latencies it is necessary to create `SearchSpaceLatencyContainer` and fill it with precalculated latencies:

In [ ]:
container = SearchSpaceLatencyContainer(
    latency_type='mmac',
    constant_latency=precalc_constant_latency,
    operations_latencies=precalc_operations_latencies.tolist(),
)

Then, apply this container to `SearchSpaceModel`:

In [ ]:
search_space.apply_latency_container(container)

Also, you can extract latency container from `SearchSpaceModel` and, for example, visualize it:

In [ ]:
plot_latency_heatmap(search_space.get_latency_container(), figsize=(8, 8));

## Save/Load SearchSpaceLatencyContainer

To save/load `SearchSpaceLatencyContainer` use `save_to_file`/`load_from_file` methods of `SearchSpaceLatencyContainer`:

```python
container: SearchSpaceLatencyContainer = ...
container.save_to_file('my_container')
container.load_from_file('my_container')

```